# Library Book Management System

In [2]:

''' 

1B: Library Book Management System

In this project, a Library Book Management System will be implemented with 
the front end GUI developed using Tkinter and the backend database implemented in SQLite. 
It is a single window tkinter GUI application which will 
implement CRUD( create, read, update, delete) operations on a ‘books’ database, in SQLite. 
The window will contain text widgets to accept / display book title, author, year of publication, ISBN (optional) and 
a large text area/ list box to display the records which match the search criteria.

 


'''



# Lbraries 

from importlib.resources import contents
from msilib.schema import Icon, RadioButton
from operator import iconcat
from os import stat
from sre_parse import State
from tkinter import * 

import sqlite3 
import tkinter.ttk as ttk 
import tkinter.messagebox as tkMessageBox
from turtle import bgcolor, left

from colorama import Cursor


root =Tk()
root.title('Book Management System')
root.iconbitmap('bookico.ico')
w, h = root.winfo_screenwidth(), root.winfo_screenheight()
style = ttk.Style()
root.geometry("%dx%d+0+0" % (w, h))
root.geometry("1550x800")

import sqlite3
# Methods 
def Database():
    global conn, CUrsor 
    conn=sqlite3.connect('Library_Book_Management_System.db')
    CUrsor = conn.cursor()
    CUrsor.execute("CREATE TABLE IF NOT EXISTS  Library_Books_MS(mem_id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,book_title TEXT,author TEXT,year TEXT,isbn TEXT NULL)")
    # CUrsor.execute("SELECT * FROM Library_Book")
    # a=CUrsor.fetchall()
# Book_title
# Author
# Year
# ISBN
    # conn.commit()
    # CUrsor.close()
    # return a
# print(Database())
def Create():
    if (BOOK_TITLE.get()=="" or AUTHER.get()=="" or YEAR.get()=="") :
        txt_result.config(text="Please enter all the fields",fg="red")

    else:
        Database()
        CUrsor.execute("INSERT INTO Library_Books_MS(book_title,author,year,isbn) VALUES(?,?,?,?)",(str(BOOK_TITLE.get()),str(AUTHER.get()),str(YEAR.get()),str(ISBN.get())))
        # CUrsor.execute("INSERT INTO Library_Book(firstname, lastname,gender,address,username,password) VALUES(%s, %s, %s, %s, %s, %s)",str(FIRSTNAME.get()),str(LASTNAME.get()),str(GENDER.get()),str(ADDRESS.get()),str(USERNAME.get()),str(PASSWORD.get()))

        conn.commit()
        BOOK_TITLE.set("")    
        AUTHER.set("")    
        YEAR.set("")    
        ISBN.set("")    
        CUrsor.close()
        conn.close()
        txt_result.config(text="Record Inserted! ",fg="green")

def Read():
    tree.delete(*tree.get_children())
    Database()
    CUrsor.execute("SELECT * FROM Library_Books_MS ")
    fetch = CUrsor.fetchall()
    for data in fetch:
        tree.insert('','end',values=(data[0],data[1],data[2],data[3],data[4]))
    CUrsor.close()
    txt_result.config(text="Seccessfully fetched data !",fg="green")


def Updated():
    Database()
    if YEAR.get()=="":
        txt_result.config(text="This is your gender.. ",fg="red")
    else:
        tree.delete(*tree.get_children())
        CUrsor.execute("UPDATE Library_Book SET book_title=%s, author=%s, year=%s, isbn=%s",
        (str(BOOK_TITLE.get()),str(AUTHER.get()),str(YEAR.get()),str(ISBN.get()),(int(mem_id))))
    conn.commit()
    CUrsor.execute("SELECT * FROM Library_Books_MS ORDER BY year ASC")
    fetch=CUrsor.fetchall()
    for data in fetch:
        tree.insert('','end',values=(data[0],data[1],data[2],data[3],data[4]))
    CUrsor.close()
    conn.close()
    
    BOOK_TITLE.set("")    
    AUTHER.set("")    
    YEAR.set("")    
    ISBN.set("")    
    btn_create.config(state=NORMAL)
    btn_read.config(state=NORMAL)
    btn_update.config(state=DISABLED)
    btn_delete.config(state=NORMAL)
    btn_Search.config(state=NORMAL)
    txt_result.config(text="UPDATE SUCCessfully",fg='yellow')

def OnSelected(event):
    global mem_id
    curItem = tree.focus()
    contents=(tree.item(curItem))
    selecteditem = contents['values']
    mem_id=selecteditem[0]
    BOOK_TITLE.set("")    
    AUTHER.set("")    
    YEAR.set("")    
    ISBN.set("")  
    BOOK_TITLE.set(selecteditem[1])    
    AUTHER.set(selecteditem[2])    
    YEAR.set(selecteditem[3])    
    ISBN.set(selecteditem[4])
    btn_create.config(state=DISABLED)  
    btn_read.config(state=DISABLED)
    btn_update.config(state=NORMAL)
    btn_delete.config(state=DISABLED)
    btn_Search.config(state=DISABLED)





def Search_Books_MS(cursor, title, author, year, isbn):
    '''
    give me a cursor to database, and a book, i'll search it.
    '''
    try:
        cursor.execute("SELECT * FROM books WHERE title=?", (title,))
        searched_titles = cursor.fetchall()
        cursor.execute("SELECT * FROM books WHERE author=?", (author,))
        searched_authors = cursor.fetchall()
        cursor.execute("SELECT * FROM books WHERE year=?", (year,))
        searched_years = cursor.fetchall()
        cursor.execute("SELECT * FROM books WHERE ISBN=?", (isbn,))
        searched_isbns = cursor.fetchall()
        return(searched_titles + searched_authors + searched_years + searched_isbns)
    except:
        return(-1)


def Search():
    Database()
    print("search book")
    title = BOOK_TITLE.get()
    author = AUTHER.get()
    year = YEAR.get()
    isbn = ISBN.get()
    res = Search_Books_MS(CUrsor, title, author, year, isbn)
    if res==-1:
        tkMessageBox.showerror("Error!", "something went wrong!")
    if res!=-1:
        tree.delete('0','end')
        for i in range(len(res)):
            tree.insert(i,res[i])  
    


def DeleteITEM():
    if not tree.selection():
        txt_result.config(text="Please select an item first",fg="red")
    else:
        result=tkMessageBox.askquestion('DELETE','Do you wont to delete This Item. ',icon='warning')
        if result=='yes':
            curItem = tree.focus()
            contents=(tree.item(curItem))
            selecteditem=contents['values']
            tree.delete(curItem)
            Database()
            CUrsor.execute("DELETE FROM Library_Books_MS WHERE mem_id = %d "%selecteditem[0])
            conn.commit()
            conn.close()
            txt_result.config(text="The item successfully deleted from the database!",fg='red')
    




def Exit():
    result = tkMessageBox.askquestion("EXIT",'Do you want to exit ? (y/n)', icon='warning')
    if result == 'yes':
        root.destroy()
        exit()



# Variable declareation 
BOOK_TITLE = StringVar()
AUTHER = StringVar()
YEAR = StringVar()
ISBN = StringVar()




# FRAME 



Top = Frame(root, width=900,height=120,bd=8,relief='raise',bg='#0000FF')
Top.pack(side=TOP)
Left=Frame(root,width=400,height=800,bd=8,relief='raise',bg='green')
Left.pack(side=LEFT)
Right = Frame(root,width=400,height=800,bd=8,relief='raise',bg='yellow')
Right.pack(side=RIGHT)


Forms = Frame(Left,width=300,height=600,bg='red')
Forms.pack(side=TOP)

Buttons = Frame(Left, width=300,height=100,bd=8,relief='raise')
Buttons.pack(side=BOTTOM)


# Labels 
# from PIL import Image, ImageTk
# img=PhotoImage('sunsine.jpg')
# style.configure('TEntry', )

txt_title=Label(Top,width=900,height=3,font=('monospace',24,'bold'),padx=25,text="Library Book Management System",foreground = 'white',bg='red')
txt_title.pack()
txt_booktitle=Label(Forms,text="Book Title       :",font=('arial',16),bd=15,width=22)
txt_booktitle.grid(row=0,stick="e")

txt_author=   Label(Forms,text="Author           :",font=('arial',16),bd=15,width=22)
txt_author.grid(row=1,stick="e")

txt_year=     Label(Forms,text="Year             :",font=('arial',16),bd=15,width=22)
txt_year.grid(row=2,stick="e")

txt_isbn     =Label(Forms,text="ISBN (Optional) :",font=('arial',16),bd=15,width=22)
txt_isbn.grid(row=3,stick="e")

txt_result =Label(Buttons)
txt_result.pack(side=TOP)



###################### ENTRY WIDGET##############################


style = ttk.Style()
style.configure('TEntry', foreground = 'green')
  

txt_booktitle=ttk.Entry(Forms,textvariable=BOOK_TITLE,width=30,justify = CENTER,font = ('courier', 15, 'bold'))
txt_booktitle.grid(row=0, column=1)

txt_author=ttk.Entry(Forms,textvariable=AUTHER,width=30,justify = CENTER,font = ('courier', 15, 'bold'))
txt_author.grid(row=1, column=1)

txt_year=ttk.Entry(Forms,textvariable=YEAR,width=30,justify = CENTER,font = ('courier', 15, 'bold'))
txt_year.grid(row=2, column=1)

txt_isbn=ttk.Entry(Forms,textvariable=ISBN,width=30,justify=CENTER,font = ('courier', 15, 'bold'))
txt_isbn.grid(row=3, column=1)


######################################## BUTTONS WIDGET ########################
btn_create=ttk.Button(Buttons, width=10,text="Add", command=Create)
btn_create.pack(side=LEFT)

btn_read=ttk.Button(Buttons, width=10,text="Display", command=Read)
btn_read.pack(side=LEFT)

btn_delete=ttk.Button(Buttons, width=10,text="Delete", command=DeleteITEM)
btn_delete.pack(side=LEFT)

btn_update=ttk.Button(Buttons, width=10,text="Issue", state=Read)
btn_update.pack(side=LEFT)

btn_Search=ttk.Button(Buttons, width=10,text="Search", command=Search)
btn_Search.pack(side=LEFT)

btn_exit=ttk.Button(Buttons, width=10,text="Exit", command=Exit)
btn_exit.pack(side=LEFT)



######################################### LIST WIDGET ###################################
scrollbary = Scrollbar(Right,orient=VERTICAL)
scrollbarx = Scrollbar(Right,orient=HORIZONTAL)

tree = ttk.Treeview(Right, columns=("Sr","Book Title","Auther","Year","ISBN"),height=20)

scrollbary.config(command=tree.yview)
scrollbary.pack(side=RIGHT, fill = Y)
scrollbarx.config(command=tree.xview)
scrollbarx.pack(side=BOTTOM, fill = X)

tree.heading('Sr',text="Sr",anchor=W)
tree.heading('Book Title',text="Book Title",anchor=W)
tree.heading('Auther',text="Author",anchor=W)
tree.heading('Year',text="Year",anchor=W)
tree.heading('ISBN',text="ISBN",anchor=W)
tree.column('#0',stretch=NO,minwidth=0,width=0,)
tree.column('#1',stretch=NO,minwidth=0,width=110)
tree.column('#2',stretch=NO,minwidth=0,width=210)
tree.column('#3',stretch=NO,minwidth=0,width=100)
tree.column('#4',stretch=NO,minwidth=0,width=100)
tree.pack()
tree.bind('<Double-Button-1>', OnSelected)


if __name__=='__main__':
    root.mainloop()

